In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install -U datasets

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("d0rj/dialogsum-ru", split="test")
df = pd.DataFrame(dataset)

df['dialogue_length'] = df['dialogue'].apply(lambda x: len(x.split()))
df['summary_length'] = df['summary'].apply(lambda x: len(x.split()))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
dialogue_bins = pd.cut(df['dialogue_length'], bins=range(0, 350, 25))
dialogue_counts = dialogue_bins.value_counts().sort_index()
ax1.bar(dialogue_counts.index.astype(str), dialogue_counts.values, color='#1f77b4')
ax1.set_title('Распределение длины диалогов', fontsize=12)
ax1.set_xlabel('Количество слов', fontsize=10)
ax1.set_ylabel('Частота', fontsize=10)
ax1.tick_params(axis='x', rotation=45)

summary_bins = pd.cut(df['summary_length'], bins=range(0, 100, 10))
summary_counts = summary_bins.value_counts().sort_index()
ax2.bar(summary_counts.index.astype(str), summary_counts.values, color='#ff7f0e')
ax2.set_title('Распределение длины саммари', fontsize=12)
ax2.set_xlabel('Количество слов', fontsize=10)
ax2.set_ylabel('Частота', fontsize=10)
ax2.tick_params(axis='x', rotation=45)
plt.suptitle('Анализ длины текстов в датасете DialogSum-RU', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import re
import os
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    pipeline,
    BitsAndBytesConfig
)
from datasets import load_dataset
import evaluate
import pandas as pd

MODELS = {
    "llama3-8b": "meta-llama/Meta-Llama-3-8B",
    "saiga_llama3_8b": "IlyaGusev/saiga_llama3_8b",
    "fred-t5": "RussianNLP/FRED-T5-Summarizer",
    "falcon-7b": "tiiuae/falcon-7b-instruct"
}

DATASETS = ["d0rj/samsum-ru", "d0rj/dialogsum-ru"]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 1024
rouge = evaluate.load("rouge")

if not os.path.exists("summarization_results"):
    os.makedirs("summarization_results")

def load_model(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained(MODELS[model_name])
    if "t5" in model_name.lower():
        model = AutoModelForSeq2SeqLM.from_pretrained(
            MODELS[model_name],
            quantization_config=bnb_config,
            device_map="auto"
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            MODELS[model_name],
            quantization_config=bnb_config,
            device_map="auto"
        )

    return model, tokenizer

def get_prompt_template(model_name, dialog):
    if "t5" in model_name.lower():
        return f"""<LM>Вы — ассистент для написания краткого содержания диалогов. Вам будет дан диалог, и вы должны создать его краткое содержание. Содержание должно включать всю важную информацию, быть коротким и лаконичным. В нем не должно быть лишней информации или данных, не относящихся к диалогу.
Напишите краткое содержание этого диалога:
{dialog}
Краткое содержание:"""
    elif "falcon" in model_name.lower():
        return f"""Вы — ассистент для написания краткого содержания диалогов. Вам будет дан диалог, и вы должны создать его краткое содержание. Содержание должно включать всю важную информацию, быть коротким и лаконичным. В нем не должно быть лишней информации или данных, не относящихся к диалогу.
Напишите краткое содержание этого диалога:
{dialog}
Краткое содержание:"""
    else:
        return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Вы — ассистент для написания краткого содержания диалогов. Вам будет дан диалог, и вы должны создать его краткое содержание. Содержание должно включать всю важную информацию, быть коротким и лаконичным. В нем не должно быть лишней информации или данных, не относящихся к диалогу.
Напишите краткое содержание этого диалога:
{dialog}
Краткое содержание:
<|eot_id|><|start_header_id|>user<|end_header_id|>
<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

def generate_summary(model, tokenizer, model_name, dialog):
    prompt = get_prompt_template(model_name, dialog)
    if "t5" in model_name.lower():
        inputs = tokenizer(prompt, return_tensors="pt", max_length=MAX_LENGTH, truncation=True).to(DEVICE)
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )
    else:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=MAX_LENGTH, truncation=True).to(DEVICE)
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def evaluate_summary(reference, prediction, model_name):
    processed_pred = postprocess_summary(prediction, model_name)
    rouge_score = rouge.compute(
        predictions=[processed_pred],
        references=[reference],
        use_stemmer=True
    )
    return {
        "rougeL": rouge_score["rougeL"],
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "processed_summary": processed_pred
    }

def postprocess_summary(text, model_name):
    remove_self_evaluation = True,
    remove_special_tokens = True,
    deduplicate_long_texts = True,
    deduplication_threshold = 1800
    if remove_special_tokens:
        text = re.sub(r'<\|[^>]+\|>', '', text)
    if deduplicate_long_texts and len(text) > deduplication_threshold:
        sentences = re.split(r'(?<=[.!?])\s+', text)
        unique_sentences = []
        seen_sentences = set()
        for sent in sentences:
            normalized = re.sub(r'\W+', '', sent.lower())
            if normalized not in seen_sentences:
                seen_sentences.add(normalized)
                unique_sentences.append(sent)
        text = ' '.join(unique_sentences)

    return text

def main():
    model_results = {model_name: [] for model_name in MODELS.keys()}
    for model_name in MODELS.keys():
        print(f"\nЗагрузка {model_name}...")
        model, tokenizer = load_model(model_name)
        for dataset_name in DATASETS:
            print(f"Обработка датасета {dataset_name}...")
            dataset = load_dataset(dataset_name, split="test[:200]")
            for i, example in enumerate(dataset):
                dialog = example["dialogue"]
                reference = example["summary"]
                summary = generate_summary(model, tokenizer, model_name, dialog)
                metrics = evaluate_summary(reference, summary, model_name)
                model_results[model_name].append({
                    "model": model_name,
                    "dataset": dataset_name,
                    "rougeL": metrics["rougeL"],
                    "rouge1": metrics["rouge1"],
                    "rouge2": metrics["rouge2"],
                    "summary": metrics["processed_summary"],
                    "reference": reference
                })

        df = pd.DataFrame(model_results[model_name])
        filename = f"summarization_results/{model_name}_results.csv"
        df.to_csv(filename, index=False)

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

df = pd.read_csv('falcon-7b_summarization_results.csv')
df[['rougeL', 'rouge1', 'rouge2']] = df[['rougeL', 'rouge1', 'rouge2']].apply(pd.to_numeric, errors='coerce')

first_200 = df.head(200)
avg_first_200 = first_200[['rougeL', 'rouge1', 'rouge2']].mean()

next_400 = df.iloc[200:400]
avg_next_400 = next_400[['rougeL', 'rouge1', 'rouge2']].mean()

print(avg_first_200)
print(avg_next_400)